# Combing data from multiple days / mice / conditions to create bigger data sets

In [1]:
import sys
sys.path.append('/Users/shayneufeld/GitHub/mouse_bandit/data_preprocessing_code')
sys.path.append('/Users/shayneufeld/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing as bp
from sklearn import preprocessing
import sys
import os
%matplotlib inline 

## load in csv files (from running exportTrials.m)

In [8]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked','Right Reward Prob','Left Reward Prob','Reward Given']

root_dir = '/Users/shayneufeld/GitHub/mouse_bandit/data/70_30_trial_data'

trial_df = []
mouse_ids = []
session_ids = []
for file in os.listdir(root_dir):
    if not file[0] == '.':
        file_name = os.path.join(root_dir,file)
        trial_df.append(pd.read_csv(file_name,names=columns))
        mouse_ids.append(file[:file.index('_')])
        session_ids.append(file[file.index('_')+1:file.index('_')+7])

## convert into 1 feature matrix

In [9]:
for i,df in enumerate(trial_df):
    
    curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_ids[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    

In [10]:
master_matrix.shape

(13833, 50)

In [11]:
master_matrix.head(30)

Mouse ID Session ID  Block Trial  Port Streak  Reward Streak  10_Port  \
0    dumble     41_ .c         11.0          8.0           -1.0      0.0   
1    dumble     41_ .c         12.0          9.0            1.0      1.0   
2    dumble     41_ .c         13.0         10.0            2.0      0.0   
3    dumble     41_ .c         14.0         10.0           -1.0      0.0   
4    dumble     41_ .c         15.0          1.0           -2.0      0.0   
5    dumble     41_ .c         16.0          2.0           -3.0      0.0   
6    dumble     41_ .c         17.0          1.0           -4.0      0.0   
7    dumble     41_ .c         18.0          2.0           -5.0      0.0   
8    dumble     41_ .c         19.0          3.0            1.0      0.0   
9    dumble     41_ .c         20.0          4.0            2.0      0.0   
10   dumble     41_ .c         21.0          5.0            3.0      0.0   
11   dumble     41_ .c         22.0          6.0            4.0      0.0   
12   dumble     41_ .c         23.0          7.0            5.0      0.0   
13   dumble     41_ .c         24.0          8.0            6.0      1.0   
14   dumble     41_ .c         25.0          9.0            7.0      1.0   
15   dumble     41_ .c         26.0         10.0           -1.0      0.0   
16   dumble     41_ .c         27.0         10.0           -2.0      0.0   
17   dumble     41_ .c         28.0         10.0           -3.0      0.0   
18   dumble     41_ .c         29.0         10.0            1.0      0.0   
19   dumble     41_ .c         30.0         10.0            2.0      0.0   
20   dumble     41_ .c         31.0         10.0            3.0      0.0   
21   dumble     41_ .c         32.0         10.0            4.0      0.0   
22   dumble     41_ .c         33.0         10.0            5.0      0.0   
23   dumble     41_ .c         34.0         10.0            6.0      0.0   
24   dumble     41_ .c         35.0          1.0            7.0      0.0   
25   dumble     41_ .c         36.0          2.0            8.0      0.0   
26   dumble     41_ .c         37.0          3.0           -1.0      0.0   
27   dumble     41_ .c         38.0          1.0           -2.0      0.0   
28   dumble     41_ .c         39.0          2.0            1.0      0.0   
29   dumble     41_ .c         40.0          3.0            2.0      0.0   

    10_Reward  10_ITI  10_trialDuration  9_Port   ...    2_trialDuration  \
0         0.0   1.845             1.073     1.0   ...              0.418   
1         0.0   2.980             1.432     0.0   ...              0.419   
2         0.0   1.688             0.618     0.0   ...              0.762   
3         1.0   2.937             0.586     0.0   ...              0.376   
4         1.0   1.437             0.333     0.0   ...              0.293   
5         1.0   1.383             1.059     0.0   ...              1.187   
6         1.0   1.241             0.555     0.0   ...              0.581   
7         0.0   2.834             0.352     0.0   ...              1.534   
8         1.0   7.093             0.418     0.0   ...              0.448   
9         0.0   2.208             0.419     0.0   ...              0.527   
10        1.0   1.658             0.762     0.0   ...              0.480   
11        1.0   3.032             0.376     0.0   ...              0.353   
12        0.0   7.667             0.293     1.0   ...              0.305   
13        0.0   2.333             1.187     1.0   ...              0.359   
14        0.0   1.596             0.581     0.0   ...              0.643   
15        0.0   2.032             1.534     0.0   ...              0.380   
16        0.0   1.941             0.448     0.0   ...              0.376   
17        1.0   1.342             0.527     0.0   ...              0.574   
18        1.0  17.019             0.480     0.0   ...              0.483   
19        1.0   7.543             0.353     0.0   ...              0.453   
20        1.0   4.269             0.305     0.0   

## Do one-hot encoding for mouse ID and session ID

In [8]:
def encode_categorical(array):
    if not (array.dtype == np.dtype('float64') or array.dtype == np.dtype('int64')) :
        return preprocessing.LabelEncoder().fit_transform(array) 
    else:
        return array

In [7]:
categorical = (master_matrix.dtypes.values != np.dtype('float64'))
master_matrix_1hot = master_matrix.apply(encode_categorical)

In [8]:
# Apply one hot endcoing
encoder = OneHotEncoder(categorical_features=categorical, sparse=False)  # Last value in mask is y
x = encoder.fit_transform(master_matrix.values)

NameError: name 'OneHotEncoder' is not defined

## Save combined feature matrix  

In [12]:
master_matrix.to_csv(os.path.join(root_dir,'master_7030_df.csv'))

In [10]:
master_matrix

Mouse ID Session ID  Block Trial  Port Streak  Reward Streak  10_Port  \
0     dumble     070716         11.0         10.0           -1.0      1.0   
1     dumble     070716         12.0         10.0           -2.0      1.0   
2     dumble     070716         13.0          1.0           -3.0      1.0   
3     dumble     070716         14.0          2.0           -4.0      1.0   
4     dumble     070716         15.0          1.0            1.0      1.0   
5     dumble     070716         16.0          2.0           -1.0      1.0   
6     dumble     070716         17.0          3.0            1.0      1.0   
7     dumble     070716         18.0          4.0            2.0      1.0   
8     dumble     070716         19.0          5.0            3.0      1.0   
9     dumble     070716         20.0          6.0            4.0      1.0   
10    dumble     070716         21.0          7.0            5.0      1.0   
11    dumble     070716         22.0          8.0            6.0      0.0   
12    dumble     070716         23.0          9.0            7.0      0.0   
13    dumble     070716         24.0         10.0            8.0      1.0   
14    dumble     070716         25.0         10.0           -1.0      1.0   
15    dumble     070716         26.0         10.0            1.0      1.0   
16    dumble     070716         27.0         10.0            2.0      1.0   
17    dumble     070716         28.0         10.0            3.0      1.0   
18    dumble     070716         29.0         10.0            4.0      1.0   
19    dumble     070716         30.0         10.0            5.0      1.0   
20    dumble     070716         31.0         10.0            6.0      1.0   
21    dumble     070716         32.0         10.0            7.0      1.0   
22    dumble     070716         33.0         10.0            8.0      1.0   
23    dumble     070716         34.0         10.0            9.0      1.0   
24    dumble     070716         35.0         10.0            0.0      1.0   
25    dumble     070716         36.0         10.0            0.0      1.0   
26    dumble     070716         37.0         10.0           -1.0      1.0   
27    dumble     070716         38.0         10.0            1.0      1.0   
28    dumble     070716         39.0         10.0            2.0      1.0   
29    dumble     070716         40.0         10.0            3.0      1.0   
..       ...        ...          ...          ...            ...      ...   
529    harry     072616         64.0         10.0           -1.0      1.0   
530    harry     072616         65.0         10.0            1.0      1.0   
531    harry     072616         66.0         10.0            2.0      1.0   
532    harry     072616         67.0         10.0            3.0      1.0   
533    harry     072616         68.0         10.0           -1.0      1.0   
534    harry     072616          0.0         10.0            1.0      1.0   
535    harry     072616          1.0         10.0           -1.0      1.0   
536    harry     072616          2.0         10.0           -2.0      1.0   
537    harry     072616          3.0         10.0            1.0      1.0   
538    harry     072616          4.0         10.0           -1.0      1.0   
539    harry     072616          5.0         10.0           -2.0      1.0   
540    harry     072616          6.0         10.0            1.0      1.0   
541    harry     072616          7.0         10.0           -1.0      1.0   
542    harry     072616          8.0         10.0           -2.0      1.0   
543    harry     072616          9.0         10.0           -3.0      1.0   
544    harry     072616         10.0         10.0           -4.0      1.0   
545    harry     072616         11.0         10.0           -5.0      1.0   
546    harry     072616         12.0         10.0           -6.0      1.0   
547    harry     072616         13.0         10.0            1.0      1.0   
548    harry     072616         14.0         10.0           -1.0      1.0   
